In [1]:
# import libraries
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from itertools import product
from tqdm import tqdm
from datetime import datetime, timedelta

In [2]:
collected_data = pd.read_csv('data/collected_data.csv') # Read the CSV file
data = collected_data.copy()
# Setting column 'date' as index.
data = data.set_index('date', drop=True)
del collected_data

data

,sahm,indpro,sp500,tr10,t10yff,unrate,pcepi,payems,houst,recession
date,,,,,,,,,,
1962-02-01,-0.17,0.016229,0.016139,-0.043737,1.650556,0.055,0.042,0.827913,0.396825,0
1962-03-01,-0.17,0.005350,-0.005878,-0.108990,1.078182,0.056,0.021,0.819544,0.478671,0
1962-04-01,-0.10,0.002130,-0.063973,-0.087455,1.043000,0.056,0.018,0.829109,0.518849,0
1962-05-01,-0.07,-0.001066,-0.089914,0.030636,1.441818,0.055,0.010,0.817113,0.498512,0
1962-06-01,0.00,-0.002132,-0.085381,0.035411,1.206667,0.055,0.010,0.816714,0.459325,0
...,...,...,...,...,...,...,...,...,...,...
2024-05-01,0.37,0.006954,0.046904,-0.056818,-0.847727,0.040,-0.010,0.824685,0.415179,0
2024-06-01,0.43,0.001500,0.034082,-0.177010,-1.024737,0.041,0.145,0.820780,0.422123,0
2024-07-01,0.53,-0.006214,0.011258,-0.056627,-1.081364,0.043,0.202,0.821816,0.388889,0


In [3]:
def find_best_arima_params(series):
    # Stationarity check.
    result = adfuller(series.dropna())
    d = 0 if result[1] < 0.05 else 1
    # Defining parameters for testing.
    p = range(0, 3)
    q = range(0, 3)
    pdq = list(product(p, [d], q))
    # Finding the best model.
    best_aic = np.inf
    best_params = None
    for param in pdq:
        try:
            model = ARIMA(series, order=param)
            results = model.fit()
            if results.aic < best_aic:
                best_aic = results.aic
                best_params = param
        except:
            continue
    return best_params if best_params is not None else (1,1,1)

def forecast_feature(series, periods=6):
    best_params = find_best_arima_params(series)
    model = ARIMA(series, order=best_params)
    results = model.fit()
    forecast = results.forecast(steps=periods)
    return forecast

# We find the last date with data.
last_date_with_data = pd.to_datetime(data.index[data.notna().any(axis=1)][-1])

# We create a new index for the forecast starting from the next month after the last data date.
# Important: We use pd.date_range instead of manually generating dates.
new_dates = pd.date_range(
    start=last_date_with_data + pd.offsets.MonthBegin(1),
    periods=6,
    freq='MS'  # Month Start frequency
)
new_indices = new_dates.strftime("%Y-%m-%d")

# Forecast for each column including 'recession'.
forecasted_data = pd.DataFrame(index=new_indices)
for column in tqdm(data.columns, desc="Column prediction"):
    try:
        # We only use the non-empty data for prediction.
        valid_data = data[column].dropna()
        if len(valid_data) > 0:
            forecasted_values = forecast_feature(valid_data)
            forecasted_data[column] = forecasted_values
        else:
            forecasted_data[column] = np.nan
    except Exception as e:
        print(f"Error predicting a column {column}: {str(e)}")
        forecasted_data[column] = np.nan

# Processing values ​​for 'recession'.
if 'recession' in forecasted_data.columns:
    forecasted_data['recession'] = forecasted_data['recession'].replace([np.inf, -np.inf], np.nan).fillna(-1)
    forecasted_data['recession'] = forecasted_data['recession'].apply(lambda x: 1 if x >= 0.5 else 0 if x >= 0 else -1).astype(int)

# Processing values ​​for 'recession'.
combined_data = pd.concat([data, forecasted_data])

# We sort the index to make sure all dates are in the correct order.
combined_data = combined_data.sort_index()

# Print the results.
print("\nCombined data (last 12 rows):")
print(combined_data.tail(12))

# Save the results.
combined_data.to_csv('data/combined_data.csv')
print("\nThe results are saved in 'combined_data.csv'")

# Checking the predicted data
print("\nForecast data:")
print(forecasted_data)

Column prediction:   0%|                                                                        | 0/10 [00:00<?, ?it/s]C:\Users\USER\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\USER\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\USER\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\USER\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\USER\anaconda3\Lib\site-packages\statsmodels\tsa\bas


Combined data (last 12 rows):
                sahm    indpro     sp500      tr10    t10yff    unrate  \
2024-04-01  0.370000 -0.001579 -0.042506  0.330591 -0.790909  0.039000   
2024-05-01  0.370000  0.006954  0.046904 -0.056818 -0.847727  0.040000   
2024-06-01  0.430000  0.001500  0.034082 -0.177010 -1.024737  0.041000   
2024-07-01  0.530000 -0.006214  0.011258 -0.056627 -1.081364  0.043000   
2024-08-01  0.570000  0.003373  0.022578 -0.377727 -1.459091  0.042000   
2024-09-01  0.500000 -0.002832  0.019996 -0.147409 -1.406500  0.041000   
2024-10-01  0.420318  0.000263  0.012590  0.032915 -1.212508  0.041562   
2024-11-01  0.371263  0.001896 -0.000628  0.000468 -1.054334  0.042184   
2024-12-01  0.365931  0.001896  0.012222 -0.000439 -0.907669  0.042725   
2025-01-01  0.369188  0.001896 -0.000270 -0.000439 -0.771676  0.043291   
2025-02-01  0.373684  0.001896  0.011875 -0.000439 -0.645579  0.043803   
2025-03-01  0.377939  0.001896  0.000068 -0.000439 -0.528658  0.044322   

      